## To Do

1. Some way of splitting jpeg time list into different flights
2. Need mission start and end times for each flight
3. Then match times from jpeg list with flight list to attribute each jpeg to a lat/long/alt
4. Write csv file for each flight with columns Image File Name/Lat/Long/Altitude
5. Re folder into flights, then mission and not mission data

## Set Up

1. In the directory there are all the jpegs, and also the arw images too
2. csv file with Image Names and Image Times from previous script
3. A subdirectory, let's say entitle 'FlightLogs', with all the pertinent flight logs inside (csv format)

In [56]:
with open('jpgImageTimes.csv') as jpgFile:
    jpgFileLines = jpgFile.read().split("\n")
    jpgImageNames = [(jpgFileLines[x]).split(',')[0] for x in range(0, len(jpgFileLines)-1)]
    jpgImageTimes = [(jpgFileLines[x]).split(',')[1] for x in range(0, len(jpgFileLines)-1)]
       
def howManySecs(clockFace):
    secs = int(clockFace[-2:]) + 60*int(clockFace[3:5]) + 60*60*int(clockFace[:2])
    return secs

jpgImageSeconds = []
for j in range(1, len(jpgFileLines)-1):
    jpgImageSeconds.append(howManySecs(jpgImageTimes[j]))

with open('jpgImageTimes2.csv', 'w') as jpgFile:
    flightNum = 'Flight'
    flightCount = 1
    flightTriggerTime = 120
    firstRow = jpgImageNames[0] + "," + jpgImageTimes[0] + "," + "Flight" + "," + "Seconds" + "\n"
    jpgFile.write(firstRow)
    secondRow = jpgImageNames[1] + "," + jpgImageTimes[1] + "," + flightNum + str(flightCount) + "," + str(jpgImageSeconds[0]) + "\n"
    jpgFile.write(secondRow)
    for i in range(1, len(jpgFileLines)-2):
        if jpgImageSeconds[i] - jpgImageSeconds[i-1] < flightTriggerTime:
            row = jpgImageNames[i+1] + "," + jpgImageTimes[i+1] + "," + flightNum +  str(flightCount) + "," + str(jpgImageSeconds[i]) + "\n"
            jpgFile.write(row)
        else:
            flightCount = flightCount + 1
            row = jpgImageNames[i+1] + "," + jpgImageTimes[i+1] + "," + flightNum +  str(flightCount) + "," + str(jpgImageSeconds[i]) + "\n"
            jpgFile.write(row)
        
print("All done :)")        

All done :)


In [70]:
flightLogDirectory = 'C:/Users/Evan/Anaconda3/.ipynb_checkpoints/JPGTest/'

import os

fileHolder = []
fileCounter = 0
for file in os.listdir(flightLogDirectory):
    if file.endswith(".csv"):
        fileHolder.append(os.path.join(flightLogDirectory, file))
        print(file)
        fileCounter = fileCounter + 1

fileCount = 0        
for fileName in fileHolder:
    with open(fileName) as flightFile:      
        flightFileLines = flightFile.read().split("\n")
    
    camShotLine = []
    word = 'SUCCESSED SIMPLE_SHOT'
   
    for i,line in enumerate(flightFileLines):
        if word in line: # or word in line.split() to search for full words
            camShotLine.append(i)
        
    missionRangeLine = []
    startWord = '[L-MIS]WP Mission: received idle vel cmd 3'
    endWord = '[L-OSD]Mission finished. now I tell app:'
   
    for i,line in enumerate(flightFileLines):
        if startWord in line: # or word in line.split() to search for full words
            missionRangeLine.append(i)
            missStartLine = i
#            print("Word \"{}\" found in line {}".format(startWord, i))

    for j,line in enumerate(flightFileLines):
        if endWord in line: # or word in line.split() to search for full words
            missionRangeLine.append(j)
            missEndLine = j
#            print("Word \"{}\" found in line {}".format(endWord, j))
             
    potMatch = len(camShotLine)
    potMatchLines = [flightFileLines[x] for x in camShotLine]

    dateWord = '|#SiF%|2'

    times = []

    for y in range(potMatch):
        charDateWord = potMatchLines[y][(potMatchLines[y].find(dateWord)+7):((potMatchLines[y].find(dateWord)+15))]
        if potMatchLines[y].find(charDateWord+".0")>1:
            times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".0")+9):((potMatchLines[y].find(charDateWord+".0")+17))])
        elif potMatchLines[y].find(charDateWord+".1")>1:  
            times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".1")+9):((potMatchLines[y].find(charDateWord+".1")+17))])
        elif potMatchLines[y].find(charDateWord+".2")>1:
            times.append(potMatchLines[y][(potMatchLines[y].find(charDateWord+".2")+9):((potMatchLines[y].find(charDateWord+".2")+17))])
        
    flightTimes = [(flightFileLines[x]).split(',')[1] for x in camShotLine]

    missStartFlightTime = (flightFileLines[missStartLine]).split(',')[1]
    missFinishFlightTime = (flightFileLines[missEndLine]).split(',')[1]
    flightTimeDiffs = [float(missStartFlightTime)-float(flightTimes[0]), float(missFinishFlightTime)-float(flightTimes[0])]
    realFlightTimes = [0,0]
    for j in range(2):
        timeHolder = (times[0]).split(':')
        for i in range(3):
            timeHolder[i] = float(timeHolder[i])
        timeHolder[2] = timeHolder[2] + flightTimeDiffs[j]
        while timeHolder[2] >= 60:
            timeHolder[2] = timeHolder[2] - 60
            timeHolder[1] = timeHolder[1] + 1
            while timeHolder[1] >= 60:
                timeHolder[1] = timeHolder[1] - 60
                timeHolder[0] = timeHolder[0] + 1
        while timeHolder[2] <= 0:
            timeHolder[2] = timeHolder[2] + 60
            timeHolder[1] = timeHolder[1] - 1
            while timeHolder[1] <= 0:
                timeHolder[1] = timeHolder[1] + 60
                timeholder[0] = timeHolder[0] - 1
        realFlightTimes[j] = timeHolder
 
    offsetTimes = [(flightFileLines[x]).split(',')[1] for x in range(missionRangeLine[0], missionRangeLine[1]+1)]
    longitudes = [(flightFileLines[x]).split(',')[3] for x in range(missionRangeLine[0], missionRangeLine[1]+1)]
    latitudes = [(flightFileLines[x]).split(',')[4] for x in range(missionRangeLine[0], missionRangeLine[1]+1)]
    altitudes = [(flightFileLines[x]).split(',')[10] for x in range(missionRangeLine[0], missionRangeLine[1]+1)]

    startSecond = 60*60*realFlightTimes[0][0] + 60*realFlightTimes[0][1] + realFlightTimes[0][2]
    endSecond = 60*60*realFlightTimes[1][0] + 60*realFlightTimes[1][1] + realFlightTimes[1][2]
    initialOffsetTime = offsetTimes[0]
    
    if fileCount == 0:
        with open('jpgFlightFile.csv', 'w') as outputFlightFile:
            firstRow = "Seconds" + "," + "Longitude" + "," + "Latitude" + "," + "Altitude"
            outputFlightFile.write(firstRow)
            
    with open('jpgFlightFile.csv', 'w') as outputFlightFile:
        for d in range(len(jpgDataLines)-1):
            if d != len(jpgDataLines)-2:
                row = jpgDataLines[d] + "\n"
                outputFlightFile.write(row)
            else:
                row = jpgDataLines[d]
                outputFlightFile.write(row)
        outputFlightFile.write("\n")
        for i in range(missionRangeLine[1]-missionRangeLine[0]+1):  #I took out the +/- 20 buffer here, maybe erroneously
            addedTime = float(offsetTimes[i]) - float(initialOffsetTime)
            timeInSeconds = startSecond + addedTime
            row =  str(timeInSeconds) + "," + longitudes[i] + "," + latitudes[i] + "," + altitudes[i] + "\n"
            outputFlightFile.write(row)
            
    fileCount = fileCount + 1
    print("Completed:", fileName)

FLY380.csv
FLY381.csv
Completed: C:/Users/Evan/Anaconda3/.ipynb_checkpoints/JPGTest/FLY380.csv
Completed: C:/Users/Evan/Anaconda3/.ipynb_checkpoints/JPGTest/FLY381.csv


In [65]:
with open('jpgFlightFile.csv', 'r') as inputFlightFile:
        missionLines = inputFlightFile.read().split("\n")
        
with open('jpgImageTimes2.csv', 'r') as inputjpgFile:
        jpgLines = inputjpgFile.read().split("\n")
        
missionSeconds = [(missionLines[x]).split(',')[0] for x in range(1, len(missionLines)-1)]
jpgSeconds = [(jpgLines[x]).split(',')[3] for x in range(1, len(jpgLines)-1)]
longitudes = [(missionLines[x]).split(',')[1] for x in range(1, len(missionLines)-1)]
latitudes = [(missionLines[x]).split(',')[2] for x in range(1, len(missionLines)-1)]
altitudes = [(missionLines[x]).split(',')[3] for x in range(1, len(missionLines)-1)]
imageNames = [(jpgLines[x]).split(',')[0] for x in range(1, len(jpgLines)-1)]

cnt = 0
jpgSecondsLineNum = []
for sec in jpgSeconds:
    lengthSec = len(sec) 
    while sec != missionSeconds[cnt][:lengthSec]:
        cnt = cnt + 1
    jpgSecondsLineNum.append(cnt)
    
with open('jpgImageFinalData.csv', 'w') as jpgFinalDataFile:
    firstRow = "File Name" + "," + "Longitude" + "," + "Latitude" + "," + "Altitude" + "\n"
    jpgFinalDataFile.write(firstRow)
    for i in range(len(jpgSeconds)-1):
        lineNum = jpgSecondsLineNum(i)
        row = imageNames[i] + "," + longitudes[lineNum] + "," + latitudes[lineNum] + "," + altitudes[lineNum] + "\n"
        jpgFinalDataFile.write(row)

5


IndexError: list index out of range